# 02 - Prompt Template - Inline

## Configuration

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using DotNetEnv;
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");

Console.WriteLine($"Configuration loaded...");


Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.38.0

Configuration loaded...


## Build Kernel


In [ ]:
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: apiKey, 
    endpoint: endpoint, 
    deploymentName: chatCompletionDeployment
);
Kernel kernel = kernelBuilder.Build();

Console.WriteLine($"Kernel created...");

Kernel created...


### Define Prompt Template Inline 

In [ ]:
string prompt = @"
    You provide information about sport events!
    You provide the name of the winning team and the score!
    You just provide the winning team name followed by a semicolon and the score.

    The {{ $SportEvent }} was won by the {{ $WinningTeam }} with a score of {{ $Score }}.

    Who won the the {{ $SportEvent }}?
";

KernelPromptTemplateFactory templateFactory = new KernelPromptTemplateFactory();
PromptTemplateConfig promptTemplateConfig = new PromptTemplateConfig(prompt);
IPromptTemplate promptTemplate = templateFactory.Create(promptTemplateConfig);

string sportEvent = "Super Sports Championship 2025";
string winningTeam = "Munich Flying Dolphins";
string score = "30-27";

KernelArguments kernelArguments = new KernelArguments() { 
    ["SportEvent"] = sportEvent,
    ["WinningTeam"] = winningTeam,
    ["Score"] = score 

};
string renderedPrompt = await promptTemplate.RenderAsync(kernel, kernelArguments);

Console.WriteLine($"Prompt template created...");
Console.WriteLine($"Prompt rendered...");

Prompt template created...
Prompt rendered...


### Create / Execute Kernel Function

In [ ]:
KernelFunction kernelFunction = kernel.CreateFunctionFromPrompt(renderedPrompt);
FunctionResult functionResult = await kernelFunction.InvokeAsync(kernel, kernelArguments);

Console.WriteLine($"Model response: \r\t {functionResult}");


	 Munich Flying Dolphins; 30-27
